
> Copyright 2023 Bruno César Brito Sant'Anna <mail@bsantanna.me>
>
> Licensed under the Apache License, Version 2.0 (the "License");
> you may not use this file except in compliance with the License.
> You may obtain a copy of the License at
>
>     http://www.apache.org/licenses/LICENSE-2.0
>
> Unless required by applicable law or agreed to in writing, software
> distributed under the License is distributed on an "AS IS" BASIS,
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
> See the License for the specific language governing permissions and
> limitations under the License.



In [15]:
from azureml.core import Workspace, Environment, Experiment, ScriptRunConfig
from azureml.train.hyperdrive import HyperDriveConfig, MedianStoppingPolicy, PrimaryMetricGoal, GridParameterSampling, \
  choice

In [16]:
# workspace settings
ws = Workspace.from_config()
ml_env = Environment.get(ws, name='AzureML-tensorflow-2.11-cuda11')
compute_target = ws.compute_targets['cpu-compute-cluster']

In [17]:
# experiment settings
script = ScriptRunConfig(
  source_directory='iban-validator-model',
  script='main-entrypoint.py',
  environment=ml_env,
  compute_target=compute_target
)

early_termination_policy = MedianStoppingPolicy(
  evaluation_interval=10, delay_evaluation=50
)

ne = [500] * 4
ia = ['relu'] * 4
oa = ['softmax'] * 4
op = ['adam'] * 4
lo = ['categorical_crossentropy'] * 4
param_sampling = GridParameterSampling({
  'batch-size': choice(8, 16, 32, 64),
  'num-epochs': choice(ne),
  'input-activation': choice(ia),
  'output-activation': choice(oa),
  'optimizer': choice(op),
  'loss': choice(lo),
})

hd_config = HyperDriveConfig(
  policy=early_termination_policy,
  run_config=script,
  hyperparameter_sampling=param_sampling,
  primary_metric_name="accuracy",
  primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
  max_total_runs=50,
  max_concurrent_runs=4
)

exp = Experiment(ws, 'iban-validator-model-experiment')

In [ ]:
# experiment execution
run = exp.submit(hd_config)
print(run.get_portal_url())
run.wait_for_completion(show_output=True)

https://ml.azure.com/runs/HD_fd8a2ec9-b5cd-42b7-a99e-66b3ba6b6ab0?wsid=/subscriptions/0d64c10a-a916-4b57-a110-e8573d336df4/resourcegroups/market-performance-eu-we/workspaces/mp-ml-workspace&tid=0de756ee-8c66-424e-89f7-7b0b1483eb80
RunId: HD_fd8a2ec9-b5cd-42b7-a99e-66b3ba6b6ab0
Web View: https://ml.azure.com/runs/HD_fd8a2ec9-b5cd-42b7-a99e-66b3ba6b6ab0?wsid=/subscriptions/0d64c10a-a916-4b57-a110-e8573d336df4/resourcegroups/market-performance-eu-we/workspaces/mp-ml-workspace&tid=0de756ee-8c66-424e-89f7-7b0b1483eb80

Streaming azureml-logs/hyperdrive.txt

[2023-07-15T17:59:31.145864][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-07-15T17:59:31.7991424Z][SCHEDULER][INFO]Scheduling job, id='HD_fd8a2ec9-b5cd-42b7-a99e-66b3ba6b6ab0_0' 
[2023-07-15T17:59:31.9543423Z][SCHEDULER][INFO]Scheduling job, id='HD_fd8a2ec9-b5cd-42b7-a99e-66b3ba6b6ab0_1' 
[2023-07-15T17:59:32.0350453Z][SCHEDULER][INFO]Scheduling job, id='HD_fd8a2ec9-b5cd-42b7-a99e-66b3ba6b6ab0_2' 
[2023-0